In [1]:
from modAL.models import ActiveLearner
from modAL.uncertainty import uncertainty_sampling
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split


# CSV 파일 경로
labeled_data_csv = "./origin_data/train.csv" # 레이블이 있는 데이터 파일
unlabeled_data_csv = "./data/unlabeld_data.csv"  # 레이블이 없는 데이터 파일
test_csv = "./origin_data/test.csv"

# CSV 파일 읽기
labeled_data = pd.read_csv(labeled_data_csv).drop(columns=["LogD", "Molecular_PolarSurfaceArea"])
unlabeled_data = pd.read_csv(unlabeled_data_csv)
test_data = pd.read_csv(test_csv).drop(columns=["LogD", "Molecular_PolarSurfaceArea"])

# 레이블이 있는 데이터 추출
X_labeled = labeled_data.drop(columns=['MLM', 'HLM'])
Y_labeled = labeled_data[['MLM', 'HLM']]

# 레이블이 없는 데이터 추출
X_unlabeled = unlabeled_data

# 초기 학습 데이터 선택 (랜덤으로 선택)
initial_idx = np.random.choice(range(len(X_labeled)), size=10, replace=False)
X_initial = X_labeled.iloc[initial_idx]
Y_initial = Y_labeled.iloc[initial_idx]

# 초기 모델 학습
learner = ActiveLearner(
    estimator=RandomForestRegressor(),
    query_strategy=uncertainty_sampling,
    X_training=X_initial,
    y_training=Y_initial
)

# Active Learning 반복 과정
n_queries = 20
for i in range(n_queries):
    query_idx, query_instance = learner.query(X_unlabeled)
    learner.teach(X_unlabeled.iloc[query_idx], Y_unlabeled.iloc[query_idx])
    
    # 쿼리된 데이터는 학습 데이터에서 제거
    X_unlabeled = X_unlabeled.drop(X_unlabeled.index[query_idx])
    Y_unlabeled = Y_unlabeled.drop(Y_unlabeled.index[query_idx])

Y_test_pred = learner.predict(test_data)



ModuleNotFoundError: No module named 'modAL'

In [9]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

# CSV 파일 경로
labeled_data_csv = "./origin_data/train.csv"  # 레이블이 있는 데이터 파일
unlabeled_data_csv = "./data/unlabeld_data.csv"  # 레이블이 없는 데이터 파일
test_csv = "./origin_data/test.csv"

# CSV 파일 읽기
labeled_data = pd.read_csv(labeled_data_csv).drop(columns=["id", "SMILES", "LogD", "Molecular_PolarSurfaceArea"])
unlabeled_data = pd.read_csv(unlabeled_data_csv).drop(columns=["SMILES"])
test_data = pd.read_csv(test_csv).drop(columns=["id", "SMILES", "LogD", "Molecular_PolarSurfaceArea"])

labeled_data = labeled_data.fillna(0)
unlabeled_data = unlabeled_data.fillna(0)
test_data = test_data.fillna(0)

# 레이블이 있는 데이터 추출
X_labeled = labeled_data.drop(columns=['MLM', 'HLM'])
Y_labeled = labeled_data[['MLM', 'HLM']]

# 레이블이 없는 데이터 추출
X_unlabeled = unlabeled_data

# 초기 학습 데이터 선택 (랜덤으로 선택)
initial_idx = np.random.choice(range(len(X_labeled)), size=10, replace=False)
X_initial = X_labeled.iloc[initial_idx]
Y_initial = Y_labeled.iloc[initial_idx]

# 초기 모델 학습 (랜덤 포레스트 회귀 모델 사용)
initial_model = RandomForestRegressor()
initial_model.fit(X_initial, Y_initial)


RandomForestRegressor()

In [11]:
# Active Learning 반복 과정
n_queries = 20
for i in range(n_queries):
    # 예측 오차를 계산하여 불확실성을 측정
    Y_pred = initial_model.predict(X_unlabeled)
    uncertainty = np.abs(Y_pred - np.mean(Y_pred, axis=0))
    
    # 불확실성이 가장 큰 데이터 포인트 선택
    query_idx = np.argmax(uncertainty)
    
    # query_idx가 현재 데이터프레임의 인덱스 범위 내에 있는지 확인
    if query_idx >= len(X_unlabeled):
        break
    
    # 쿼리된 데이터는 학습 데이터에 추가
    X_query = X_unlabeled.iloc[query_idx:query_idx+1]
    Y_query = Y_labeled.iloc[query_idx:query_idx+1]

    X_labeled = pd.concat([X_labeled, X_query])
    Y_labeled = pd.concat([Y_labeled, Y_query])
    
    # 쿼리된 데이터는 레이블이 없는 데이터에서 제거
    X_unlabeled = X_unlabeled.drop(X_unlabeled.index[query_idx])

    # 새로운 모델을 학습하여 불확실성 계산에 사용
    initial_model = RandomForestRegressor()
    initial_model.fit(X_labeled, Y_labeled)

Y_test_pred = initial_model.predict(test_data)


In [13]:

# predict
df_submission = pd.read_csv("./origin_data/sample_submission.csv")
df_submission["MLM"] = Y_test_pred[:,0]
df_submission["HLM"] = Y_test_pred[:,1]
df_submission.to_csv("./submission/submission_SS_LBW.csv", index = False, encoding = "utf-8-sig")